In [1]:
from langchain_core.tools import tool
from langchain_ollama import ChatOllama
from langchain_community.tools import DuckDuckGoSearchResults
from langgraph.prebuilt import ToolNode
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper

wrapper = DuckDuckGoSearchAPIWrapper(max_results=3)

search = DuckDuckGoSearchResults(api_wrapper=wrapper, output_format='list')



model = ChatOllama(model="jacob-ebey/phi4-tools:latest")


@tool
def magic_function(input: int) -> int:
    """Applies a magic function to an input."""
    return input + 2


tools = [search]


query = "what should be the required materials and experiment procedure of the following lab Experiment (To verify the relationship between voltage (V), current (I), and resistance (R) in an electrical circuit, as expressed by Ohm's Law: V=IRV = IRV=IR)"

In [2]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", '''You are a Lab Assistant designed to assist with scientific experiments. Your task is to provide:

Experiment Materials: A list of required materials and equipment.
Experiment Steps: A detailed step-by-step guide for conducting the experiment.
Safety Procedures: Essential precautions to ensure a safe experiment.
Before generating the final response, you will:

Search the query using DuckDuckGo to gather up-to-date and relevant information.
Analyze the results to determine if the information is complete.
Enhance the response by integrating the best available knowledge before providing the final answer.'''),
        ("human", "{input}"),
        # Placeholders fill up a **list** of messages
        ("placeholder", "{agent_scratchpad}"),
    ]
)


agent = create_tool_calling_agent(model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

for step in agent_executor.stream({"input": query}):
    print(step)

{'actions': [ToolAgentAction(tool='duckduckgo_results_json', tool_input={'query': "Ohm's Law experiment materials and procedure"}, log='\nInvoking: `duckduckgo_results_json` with `{\'query\': "Ohm\'s Law experiment materials and procedure"}`\n\n\n', message_log=[AIMessageChunk(content='', additional_kwargs={}, response_metadata={'model': 'jacob-ebey/phi4-tools:latest', 'created_at': '2025-02-14T05:21:53.096165291Z', 'done': True, 'done_reason': 'stop', 'total_duration': 5632217657, 'load_duration': 2906323310, 'prompt_eval_count': 335, 'prompt_eval_duration': 266000000, 'eval_count': 157, 'eval_duration': 2210000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-f95f4e22-81ae-4b0e-a989-6a155d107dbb', tool_calls=[{'name': 'duckduckgo_results_json', 'args': {'query': "Ohm's Law experiment materials and procedure"}, 'id': '35ccc5b1-e29a-425b-923d-5ded850016cf', 'type': 'tool_call'}], usage_metadata={'input_tokens': 335, 'output_tokens': 157, 'tot

In [3]:
from langgraph.prebuilt import create_react_agent

langgraph_agent_executor = create_react_agent(model, tools)


messages = langgraph_agent_executor.invoke({"messages": [("human", query)]})
{
    "input": query,
    "output": messages["messages"][-1].content,
}

{'input': 'what is the value of magic_function(3)?',
 'output': 'The value of magic_function(3) is 5.'}

In [4]:
message_history = messages["messages"]

new_query = "Pardon?"

messages = langgraph_agent_executor.invoke(
    {"messages": message_history + [("human", new_query)]}
)
{
    "input": new_query,
    "output": messages["messages"][-1].content,
}

{'input': 'Pardon?',
 'output': 'The result returned by the `magic_function` when applied to the input value of 3 is 5. If you were expecting a different outcome or need further clarification, please let me know!'}

In [5]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Respond only in Spanish."),
        ("human", "{input}"),
        # Placeholders fill up a **list** of messages
        ("placeholder", "{agent_scratchpad}"),
    ]
)


agent = create_tool_calling_agent(model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

agent_executor.invoke({"input": query})

{'input': 'what is the value of magic_function(3)?',
 'output': 'El valor de la función mágica con entrada 3 es 5.'}

In [29]:
from pydantic import BaseModel,Field

class JsonnedOutput(BaseModel):
    title:str =Field(description="Title of the experiment")
    content:str = Field(description="the complete output of the agent plus ensuring that the response contains materials reuired, Experiments Steps and safety precautions")

In [4]:
from langchain_core.messages import SystemMessage
from langgraph.prebuilt import create_react_agent

system_message = '''You are a Lab Assistant designed to assist with scientific experiments. Your task is to provide:

Experiment Materials: A list of required materials and equipment.
Experiment Steps: A detailed step-by-step guide for conducting the experiment.
Safety Procedures: Essential precautions to ensure a safe experiment.
Before generating the final response, you will:

Search the query using DuckDuckGo to gather up-to-date and relevant information.
Analyze the results to determine if the information is complete.
Enhance the response by integrating the best available knowledge before providing the final answer.which is not a summary but well explained'''
# This could also be a SystemMessage object
# system_message = SystemMessage(content="You are a helpful assistant. Respond only in Spanish.")

langgraph_agent_executor = create_react_agent(model, tools, prompt=system_message)


messages = langgraph_agent_executor.invoke({"messages": [("user", query)]})

In [6]:
messages['messages']

[HumanMessage(content="what should be the required materials and experiment procedure of the following lab Experiment (To verify the relationship between voltage (V), current (I), and resistance (R) in an electrical circuit, as expressed by Ohm's Law: V=IRV = IRV=IR)", additional_kwargs={}, response_metadata={}, id='b3c57be0-1673-4c6b-bf01-8f5896ed9c78'),
 AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'jacob-ebey/phi4-tools:latest', 'created_at': '2025-02-14T05:22:35.615594999Z', 'done': True, 'done_reason': 'stop', 'total_duration': 3097766571, 'load_duration': 9775914, 'prompt_eval_count': 343, 'prompt_eval_duration': 66000000, 'eval_count': 211, 'eval_duration': 3018000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-b71eaee2-ded2-4882-b231-ad22a9c18efb-0', tool_calls=[{'name': 'duckduckgo_results_json', 'args': {'query': "Ohm's Law experiment procedure and materials"}, 'id': '5bdea9ed-b984-4517-961a-d926c34428ed

In [9]:
experiment_name = [
    "To verify the relationship between voltage (V), current (I), and resistance (R) in an electrical circuit, as expressed by Ohm's Law: V=IRV = IRV=IR",
    "To apply Kirchhoff’s Voltage Law (KVL) and Kirchhoff’s Current Law (KCL) to analyze and validate simple electrical circuits.",
    "To study the behavior of resistors in series and parallel configurations, including the equivalent resistance calculation.",
    "To study the forward and reverse bias characteristics of a PN junction diode.",
    "To analyze voltage regulation using a Zener diode.",
    "To construct and analyze the performance of half-wave and full-wave rectifiers, both with and without filters.",
    "To analyze and implement wave-shaping circuits using diodes for clipping and clamping applications.",
    "To analyze the input and output characteristics of Bipolar Junction Transistors (BJTs) and Field-Effect Transistors (FETs).",
    "To design a common emitter amplifier and analyze its frequency response.",
    "To implement and analyze operational amplifier (Op-Amp) circuits: inverting, non-inverting, summing, and differentiator configurations.",
    "To verify the operation of basic logic gates: AND, OR, NOT, NAND, NOR, XOR, and XNOR",
    "To investigate the charging and discharging behavior of a capacitor in an RC circuit and understand the time constant.",
    "To investigate the resonance behavior of an LC circuit and measure the resonant frequency.",
    "To calibrate an oscilloscope for accurate measurements of voltage and time.",
    "To determine the turns ratio of a transformer and verify the relationship between the primary and secondary voltages.",
    "To construct and analyze a bridge rectifier circuit and compare its performance with a half-wave rectifier.",
    "To study the frequency response of a low-pass filter and determine its cutoff frequency.",
    "To measure the input and output impedances of an operational amplifier (Op-Amp) in a given configuration.",
    "To generate and analyze Lissajous figures using an oscilloscope by applying two sinusoidal signals with different frequencies.",
    "To study the magnetic field produced by a solenoid and verify the relationship between current and magnetic field strength."
]

In [ ]:
def run_graph(message: dict):
    while True:
        try:
            return 

In [ ]:
import json
from tqdm import tqdm
message_len_list = []
for i in tqdm(range(10),desc="sample No"):
    fname = f"deepseekReAct_Final_{i}"
    for exp in tqdm(experiment_name,desc="experiment"):
        query = f"what should be the required materials and experiment procedure of the following lab Experiment ({exp})?"
        langgraph_agent_executor = create_react_agent(model, tools, prompt=system_message)
        
        messages = langgraph_agent_executor.invoke({"messages": [("user", query)]})
        # print(messages)
        message_len_list.append(len(messages['messages']))
        try:
            with open("testing_phi4/"+fname+".json","r",encoding='utf-8') as F:
                existing_data = json.load(F)
        except (FileNotFoundError, json.JSONDecodeError):
            existing_data = {"experiments": []}  # Default structure if file is empty or missing
        json_output = {exp:messages['messages'][-1].content}
        existing_data["experiments"].append(json_output)
        with open("testing_phi4/"+fname+".json","w",encoding='utf-8') as F:
            json.dump(existing_data,F,indent=4)
        

sample No:  90%|█████████ | 9/10 [55:07<06:07, 367.54s/it]


RemoteProtocolError: peer closed connection without sending complete message body (incomplete chunked read)

In [14]:
count = 0
for i in message_len_list:
    if i ==2:
        count+=1

In [15]:
count

1

Based on the gathered information, here is a detailed guide for conducting an experiment to verify Ohm's Law:

### Experiment Materials:
1. **Voltmeter**: For measuring voltage across the resistor.
2. **Ammeter**: For measuring current flowing through the circuit.
3. **Resistor(s)**: The conductor (such as nichrome wire) with known resistance values.
4. **Dry Cells or Battery Eliminator**: To provide a constant voltage source.
5. **Connecting Wires**: For connecting all components in the circuit.
6. **Key/ Switch**: To control the flow of current in the circuit.
7. **Rheostat (Optional)**: To vary the resistance and observe changes in current and voltage.

### Experiment Steps:
1. **Circuit Setup**:
   - Assemble a simple series circuit with a voltmeter connected in parallel across the resistor to measure the potential difference (V) and an ammeter connected in series to measure the current (I).
   - Connect the resistor, dry cells or battery eliminator, rheostat if available, key/switch, and connecting wires.

2. **Initial Measurement**:
   - Before starting measurements, make sure all connections are secure.
   - Set up the circuit such that the switch is open initially to avoid any current flow.

3. **Data Collection**:
   - Close the switch to complete the circuit.
   - Record the voltage (V) and current (I) from the voltmeter and ammeter, respectively.
   - If using a rheostat, vary its resistance to change the overall circuit resistance and record multiple sets of V and I values.

4. **Repeat for Different Conditions**:
   - Repeat step 3 by changing the voltage or resistor value if you have access to different resistors.
   - Ensure each measurement is taken accurately with the correct ranges on the voltmeter and ammeter.

5. **Calculations**:
   - Use Ohm's Law formula \( V = IR \) to calculate theoretical values of resistance (R).
   - Compare these calculated values with the actual resistor value provided.

6. **Graphing Data**:
   - Plot a graph with voltage (V) on the y-axis and current (I) on the x-axis.
   - If Ohm's Law holds true, the graph should be linear with the slope equal to the resistance of the circuit.

### Safety Procedures:
1. **Proper Handling**: Handle electrical components carefully to avoid any damage or short circuits.
2. **Switch Usage**: Always start and end the experiment with the switch open to ensure safety from continuous current flow.
3. **Voltage Range Check**: Ensure that the voltmeter and ammeter are set to the appropriate ranges before connecting them in the circuit to prevent instrument damage.
4. **Dry Hands**: Keep your hands dry when handling electrical components to avoid electric shocks.

By following these steps, you can effectively verify Ohm's Law and observe how voltage, current, and resistance interrelate in an electrical circuit.